In [1]:
"""
This script provides a training and evaluation pipeline for both Classical Graph Convolutional Networks (GCN)
and Quantum Graph Neural Networks (QGNN). It's designed to be run locally or within a qBraid Lab
environment, mimicking the parameter passing of a GitHub Actions workflow.

To run this script:
1. Ensure your 'src/synqronix/config.py' and 'src/synqronix/main.py' modules are correctly
   placed in your repository structure.
2. Install all necessary Python dependencies listed in your 'requirements.txt' file
   (e.g., qbraid, torch, dgl, qiskit, etc.).
3. Set your QBRAID_API_KEY environment variable before running the script:
   - Linux/macOS: export QBRAID_API_KEY="YOUR_ACTUAL_QBRAID_API_KEY"
   - Windows (PowerShell): $env:QBRAID_API_KEY="YOUR_ACTUAL_QBRAID_API_KEY"
   - Windows (Command Prompt): set QBRAID_API_KEY="YOUR_ACTUAL_QBRAID_API_KEY"
4. Make sure you have the necessary data files in the specified './data'.
5. Execute the script: python run_training_pipeline.py
"""


'\nThis script provides a training and evaluation pipeline for both Classical Graph Convolutional Networks (GCN)\nand Quantum Graph Neural Networks (QGNN). It\'s designed to be run locally or within a qBraid Lab\nenvironment, mimicking the parameter passing of a GitHub Actions workflow.\n\nTo run this script:\n1. Ensure your \'src/synqronix/config.py\' and \'src/synqronix/main.py\' modules are correctly\n   placed in your repository structure.\n2. Install all necessary Python dependencies listed in your \'requirements.txt\' file\n   (e.g., qbraid, torch, dgl, qiskit, etc.).\n3. Set your QBRAID_API_KEY environment variable before running the script:\n   - Linux/macOS: export QBRAID_API_KEY="YOUR_ACTUAL_QBRAID_API_KEY"\n   - Windows (PowerShell): $env:QBRAID_API_KEY="YOUR_ACTUAL_QBRAID_API_KEY"\n   - Windows (Command Prompt): set QBRAID_API_KEY="YOUR_ACTUAL_QBRAID_API_KEY"\n4. Make sure you have the necessary data files in the specified \'./data\'.\n5. Execute the script: python run_trai

In [2]:
!pip install -r requirements.txt

import os
import sys
import json
import argparse

# --- 1. Setup and Parameter Definition ---

# Ensure the src directory is in the Python path
# This is crucial if you're running the script from the repository root.
if os.path.abspath('src') not in sys.path:
    sys.path.insert(0, os.path.abspath('src'))

# Import your project-specific modules
try:
    from src.synqronix.config import define_parameters
    from src.synqronix.main import main, build_data
except ImportError as e:
    print(f"Error importing synqronix modules: {e}")
    print("Please ensure 'src/synqronix/config.py' and 'src/synqronix/main.py' exist and are accessible.")
    sys.exit(1)

print("Modules loaded successfully.")

Modules loaded successfully.


In [3]:
# --- Define Parameters ---
# These values serve as defaults if not overridden by environment variables.
# For qBraid Lab, you could potentially set these as environment variables
# in your qBraid Lab environment settings if you prefer, or pass them
# as part of a custom startup script if qBraid supports that.

NB_DATA_DIR = os.environ.get("NB_DATA_DIR", "/Users/mariayuffa/Downloads/Auditory cortex data")
NB_SAVE_DIR = os.environ.get("NB_SAVE_DIR", "checkpoints/")
NB_NUM_EPOCHS = int(os.environ.get("NB_NUM_EPOCHS", "100"))
NB_QUANTUM_DEVICE = os.environ.get("NB_QUANTUM_DEVICE", "default")

# The original workflow uses QBRAID_RUN_INPUTS, let's keep that as a fallback/override mechanism
qbraid_run_inputs_str = os.environ.get("QBRAID_RUN_INPUTS", "{}")
try:
    qbraid_run_inputs = json.loads(qbraid_run_inputs_str)
except json.JSONDecodeError:
    print(f"Warning: Could not parse QBRAID_RUN_INPUTS: {qbraid_run_inputs_str}. Using defaults.")
    qbraid_run_inputs = {}

# --- Create an argparse.Namespace object to mimic 'args' ---
# We'll use the define_parameters to get a base parser, then populate args
parser = define_parameters()
args = parser.parse_args([])

# Create an empty namespace to hold our arguments
args = argparse.Namespace()

# Populate with defaults from define_parameters first
# This ensures all expected arguments are present, even if not explicitly defined above.
default_args, _ = parser.parse_known_args([])
for arg_name in vars(default_args):
    setattr(args, arg_name, getattr(default_args, arg_name))

# Override with script-specific parameters (NB_ prefix)
args.data_dir = NB_DATA_DIR
args.save_dir = NB_SAVE_DIR
args.num_epochs = NB_NUM_EPOCHS
args.quantum_device = NB_QUANTUM_DEVICE

# Override with any QBRAID_RUN_INPUTS (highest precedence)
for k, v in qbraid_run_inputs.items():
    # Convert types if necessary, argparse usually handles this. Here we assume string for simplicity
    if k == "num_epochs":
        setattr(args, k, int(v))
    elif k == "hidden_dim": # Assuming hidden_dim might be passed from GH Actions
        setattr(args, k, int(v))
    else:
        setattr(args, k, v)


# Ensure api_key is handled. It's not in workflow_dispatch inputs, but needed for QGNN.
# It should come from the environment where the notebook is run (e.g., qBraid Lab environment variables)
args.api_key = os.environ.get("QBRAID_API_KEY", None)

print("--- Final Arguments for Pipeline --- ")
for arg_name, arg_value in vars(args).items():
    if arg_name == 'api_key' and arg_value is not None: # Don't print the actual key
        print(f"  {arg_name}: {'****' * 5}")
    else:
        print(f"  {arg_name}: {arg_value}")
print("------------------------------------ ")

--- Final Arguments for Pipeline --- 
  data_dir: /Users/mariayuffa/Downloads/Auditory cortex data
  save_dir: checkpoints/
  model_type: QGCN
  hidden_dim: 64
  num_layers: 3
  dropout_rate: 0.5
  lr: 0.001
  weight_decay: 0.0001
  batch_size: 32
  num_epochs: 100
  k: 20
  connectivity_threshold: 0.5
  add_hyperedges: False
  checkpoint_freq: 5
  resume_from: None
  evaluate_only: None
  seed: 42
  col_width: 50
  col_height: 30
  dataset: functional
  quantum_device: default
  shots: None
  api_key: None
  q_depths: [2, 2]
------------------------------------ 


In [ ]:
# --- Main Training Pipeline ---
# --- 1. Build Data Loaders and Environment ---
print("\nBuilding data and environment...")
(data_loader, train_loader, val_loader,
test_loader, device, checkpoint_dir, eval_dir) = build_data(args)

print("Data and environment built successfully.")
print(f"Device: {device}")
print(f"Checkpoint Directory: {checkpoint_dir}")
print(f"Evaluation Directory: {eval_dir}")

In [ ]:
# Comparison between the two models:
# --- 2. Train Classical GCN Model ---
print("\n--- Starting Training for Model Type: GCN ---")
args.model_type = "GCN" # Ensure model_type is set for GCN
main(args, data_loader, train_loader,
     val_loader, test_loader, device,
     checkpoint_dir, eval_dir)
print("Training of GCN completed successfully!")

# --- 3. Train Quantum GNN Model (QGNN) ---
print("\n--- Starting Training for Model Type: QGNN ---")
args.model_type = "QGNN" # Ensure model_type is set for QGNN

if not args.api_key:
     raise ValueError("QBRAID_API_KEY environment variable is not set. Please set it in your environment to run QGNN.")

main(args, data_loader, train_loader,
     val_loader, test_loader, device,
     checkpoint_dir, eval_dir)

print("Training of QGNN completed successfully!")

# --- 5. Conclusion ---
print("\nBoth GCN and QGNN training pipelines have completed. Check your specified 'save_dir' and 'eval_dir' for results.")

